In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
from financial_reports_collector import filings_collector
import datetime as dt
import requests
import pprint as pp
import xml.etree.ElementTree as ET

In [2]:
# Access page
ticker = input('ticker?').lower()

xml_filings_df = filings_collector(ticker)


ticker?gme


In [3]:
xml_filings_df = xml_filings_df.loc[xml_filings_df['xbrl_link'] != 'N/A']
xml_filings_df = xml_filings_df.reset_index(drop=True)
xml_filings_df.tail(20)

,date,doc_link,filings,xbrl_link
0,2018-12-11,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
1,2018-09-11,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
2,2018-06-12,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
3,2018-04-02,https://www.sec.gov/Archives/edgar/data/132638...,10-K,https://www.sec.gov/Archives/edgar/data/132638...
4,2017-12-05,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
5,2017-09-06,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
6,2017-06-06,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
7,2017-03-27,https://www.sec.gov/Archives/edgar/data/132638...,10-K,https://www.sec.gov/Archives/edgar/data/132638...
8,2016-12-06,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...
9,2016-09-07,https://www.sec.gov/Archives/edgar/data/132638...,10-Q,https://www.sec.gov/Archives/edgar/data/132638...


In [41]:
# lxml_ten_df = xml_filings_df.loc[xml_filings_df['filings'] == '10-Q']

lxml_ten_q = xml_filings_df['xbrl_link'][3]

xbrl_resp = requests.get(lxml_ten_q)
xbrl_str = xbrl_resp.text

root = ET.fromstring(xbrl_str)


uri_dict = {'xbrl' :'http://www.xbrl.org/2003/instance',
           'sec' : 'http://xbrl.sec.gov/dei/2014-01-31', 
           'company' : 'http://www.gamestop.com/20180203',
           'gaap' : 'http://fasb.org/us-gaap/2017-01-31'}



In [ ]:
doc_date_str = root.find('sec:DocumentPeriodEndDate', uri_dict).text
doc_date_dt = dt.date.fromisoformat(doc_date_str)

entry_dict = {'company_name' : root.find('sec:EntityRegistrantName', uri_dict).text,
            'trading_symbol' : root.find('sec:TradingSymbol', uri_dict).text,
            'CIK' : root.find('sec:EntityCentralIndexKey', uri_dict).text,
            'doc_date' : doc_date_dt,
            'doc_year' : root.find('sec:DocumentFiscalYearFocus', uri_dict).text,
            'doc_period' : root.find('sec:DocumentFiscalPeriodFocus', uri_dict).text,
            'common_shares_outstanding' : root.find('sec:EntityCommonStockSharesOutstanding', uri_dict).text,
            'amendment_flag' : root.find('sec:AmendmentFlag', uri_dict).text}

In [82]:
for child in root:
    
    uri_pos = child.tag.index('}')
    uri = ''.join(child.tag[1:uri_pos])
    name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])
    current_focus_period = root.find('sec:DocumentFiscalPeriodFocus', uri_dict).text
    current_focus_year = root.find('sec:DocumentFiscalYearFocus', uri_dict).text

#     Quater or Annual
    if current_focus_period[0] == "Q":
        current_context = str(f'{current_focus_year}{current_focus_period}QTD')
    elif current_focus_period[0] == "F":
        current_context = str(f'{current_focus_year}Q4YTD')  
        
#     Finding only current info
    if uri == uri_dict['gaap']:
        context_ref = child.attrib['contextRef']
        if len(context_ref) == 11:
            if context_ref[2:12] == current_context:
                try:
                    decimals = int(child.attrib['decimals'])
                    if decimals <= 0:
                        data = int(child.text)
                    else:
                        data = float(child.text)
                        
                    if name in entry_dict:
                        entry_dict[name].append(data)
                    else:
                        entry_dict[name] = [data]    
                except:
                    print(f'{name}')
                
        elif len(context_ref) == 8:
             if context_ref[2:8] == current_context[0:6]:
                    try:
                        decimals = int(child.attrib['decimals'])
                        if decimals <= 0:
                            data = int(child.text)
                        else:
                            data = float(child.text)
                                                
                        if name in entry_dict:
                            entry_dict[name].append(data)
                        else:
                            entry_dict[name] = [data]
                    except:
                        print(f'{name}')
                    
                    

CommonStockVotingRights
EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate
EffectiveIncomeTaxRateReconciliationChangeInDeferredTaxAssetsValuationAllowance
EffectiveIncomeTaxRateReconciliationDeductionsQualifiedProductionActivities
EffectiveIncomeTaxRateReconciliationDispositionOfBusiness
EffectiveIncomeTaxRateReconciliationForeignIncomeTaxRateDifferential
EffectiveIncomeTaxRateReconciliationNondeductibleExpenseImpairmentLosses
EffectiveIncomeTaxRateReconciliationNondeductibleExpenseOther
EffectiveIncomeTaxRateReconciliationNondeductibleExpenseShareBasedCompensationCost
EffectiveIncomeTaxRateReconciliationOtherAdjustments
EffectiveIncomeTaxRateReconciliationStateAndLocalIncomeTaxes
EffectiveIncomeTaxRateReconciliationTaxCreditsForeign
EffectiveIncomeTaxRateReconciliationTaxExemptIncome
FiniteLivedIntangibleAssetUsefulLife
SharebasedCompensationSharesAuthorizedUnderStockOptionPlansExercisePriceRangeOutstandingOptionsWeightedAverageRemainingContractualTerm2
AccountsPayabl

In [83]:
print(entry_dict)

{'company_name': 'GameStop Corp.', 'trading_symbol': 'GME', 'CIK': '0001326380', 'doc_date': datetime.date(2018, 2, 3), 'doc_year': '2017', 'doc_period': 'FY', 'common_shares_outstanding': '101583319', 'amendment_flag': 'false', 'AccountsPayableCurrent': [902000000, 902000000], 'AccrualForTaxesOtherThanIncomeTaxesCurrent': [63400000, 63400000], 'AccruedIncomeTaxesCurrent': [37500000, 37500000], 'AccruedLiabilitiesCurrent': [976100000, 976100000], 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment': [1330000000, 1330000000], 'AccumulatedOtherComprehensiveIncomeLossNetOfTax': [12200000, 12200000], 'AdditionalPaidInCapitalCommonStock': [22100000, 22100000], 'AdjustmentToAdditionalPaidInCapitalIncomeTaxEffectFromShareBasedCompensationNet': [0, 0], 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue': [25600000, 25600000], 'AdvertisingExpense': [83300000, 83300000], 'AllowanceForDoubtfulAccountsReceivableCurrent': [9400000

In [31]:
gaap_keys = []
for child in root:
    tag_list = list(child.tag)
    uri_pos = tag_list.index('}')
    uri = ''.join(tag_list[1:uri_pos])
    name = ''.join(tag_list[int(uri_pos+1):len(tag_list)])
    if uri == uri_dict['gaap']:
        if name:
            try:
                decimals = int(child.attrib['decimals'])
                if decimals <= 0:
                    data = int(child.text)
                else:
                    data = float(child.text)
                context_ref = child.attrib['contextRef']
                new_key = {'key' : name, 'context_ref' : context_ref,
                          'data' : data}
                gaap_keys.append(new_key)
            except:
                print(f'{name}')

CommitmentsAndContingencies
CommitmentsAndContingencies
CommitmentsAndContingencies
DebtInstrumentBasisSpreadOnVariableRate1
DebtInstrumentBasisSpreadOnVariableRate1
DebtInstrumentCovenantDescription
DebtInstrumentInterestRateStatedPercentage
DebtInstrumentInterestRateStatedPercentage
DebtInstrumentInterestRateStatedPercentage
FiniteLivedIntangibleAssetUsefulLife
LineOfCreditFacilityAssetRestrictions
LineOfCreditFacilityCovenantTerms
LineOfCreditFacilityDividendRestrictions
NumberOfCountriesInWhichEntityOperates
NumberOfReportableSegments
NumberOfReportableSegments
NumberOfStatesInWhichEntityOperates
NumberOfStores
BasisOfAccountingPolicyPolicyTextBlock
CommitmentsAndContingenciesDisclosureTextBlock
DebtDisclosureTextBlock
DeferredRevenueDisclosureTextBlock
DescriptionOfNewAccountingPronouncementsNotYetAdopted
DisaggregationOfRevenueTableTextBlock
DisposalGroupsIncludingDiscontinuedOperationsDisclosureTextBlock
EarningsPerShareTextBlock
FairValueDisclosuresTextBlock
GoodwillAndIntangib

In [33]:
gaap_names_df = pd.DataFrame(gaap_keys)
# gaap_names_df.to_csv(f'../Data/{ticker}_key.csv', index=False)
gaap_names_df.tail()

,context_ref,data,key
645,FD2018Q3YTD,102000000.0,WeightedAverageNumberOfDilutedSharesOutstanding
646,FD2017Q3QTD,101500000.0,WeightedAverageNumberOfSharesOutstandingBasic
647,FD2017Q3YTD,101400000.0,WeightedAverageNumberOfSharesOutstandingBasic
648,FD2018Q3QTD,102200000.0,WeightedAverageNumberOfSharesOutstandingBasic
649,FD2018Q3YTD,102000000.0,WeightedAverageNumberOfSharesOutstandingBasic


In [ ]:
for entry in root.findall('gaap:CommonStockDividendsPerShareCashPaid', uri_dict):
    print(entry.attrib)

In [ ]:
for child in root:
    if child.tag == '{http://fasb.org/us-gaap/2017-01-31}Assets':
        country_list = list(child.attrib['contextRef'])
        country_pos = len(country_list)
        country = ''.join(country_list[country_pos-2:country_pos])
        report = ''.join(country_list[0:8])
        if country == 'US':
            print(report)
            print(child.text)
        
    

In [ ]:
pprinted = pp.PrettyPrinter(indent=4)
pprinted.pprint(root.findall('{http://fasb.org/us-gaap/2017-01-31}Assets'))

In [ ]:
'dei:currentfiscalyearenddate'
'dei:entitycommonstocksharesoutstanding'
'dei:entityfilercategory'
'dei:entityregistrantname'
'dei:tradingsymbol'